In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFDistilBertModel, DistilBertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import DistilBertTokenizer
import pickle
# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load the dataset
df = pd.read_csv('F:\shaheen\shaheen\data2.csv')

df= df.sample(frac=1)

In [6]:
# Preprocess the data
df['Symptoms'] = df['Symptoms'].str.lower()
# df['Medical_History'] = df['Medical_History'].str.lower()

X_text = df['Symptoms']
y = df['Disease']

# Tokenize input data
X_tokenized = tokenizer(X_text.tolist(), padding=True, truncation=True, return_tensors='tf')

# Convert tensors to NumPy arrays
X_input_ids = X_tokenized['input_ids'].numpy()
X_attention_masks = X_tokenized['attention_mask'].numpy()

# Split the dataset into train, validation, and test sets
X_train_input_ids, X_temp_input_ids, X_train_attention_masks, X_temp_attention_masks, y_train, y_temp = train_test_split(X_input_ids, X_attention_masks, y, test_size=0.2, random_state=42)
X_val_input_ids, X_test_input_ids, X_val_attention_masks, X_test_attention_masks, y_val, y_test = train_test_split(X_temp_input_ids, X_temp_attention_masks, y_temp, test_size=0.5, random_state=42)





In [7]:
class DiseaseClassifier(tf.keras.Model):
    def __init__(self, num_classes):
        super(DiseaseClassifier, self).__init__()
        self.distilbert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.dense = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        outputs = self.distilbert(inputs)[0]
        pooled_output = outputs[:, 0, :]  # take [CLS] token
        pooled_output = self.dropout(pooled_output)
        logits = self.dense(pooled_output)
        return logits




In [8]:
# Initialize and compile the model
t=df['Disease'].unique()
print(t)
num_classes = len(df['Disease'].unique())
print(num_classes)
model = DiseaseClassifier(num_classes)

# Call the model once to create its variables
dummy_input = tf.constant([[0]], dtype=tf.int32)
_ = model(dummy_input)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])


['Diabetes' 'Asthma' 'Hypertension' 'Gastroenteritis' 'Osteoarthritis']
5


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [9]:
from sklearn.preprocessing import LabelEncoder

# Encode target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
print(y_train,y_train_encoded)
# 0-Asthma . 1-Diabetes, 2-Hypertension

2236     Osteoarthritis
669        Hypertension
1099             Asthma
1844    Gastroenteritis
920        Hypertension
             ...       
509        Hypertension
2453     Osteoarthritis
1367             Asthma
1440             Asthma
154            Diabetes
Name: Disease, Length: 1978, dtype: object [4 3 0 ... 0 0 1]


In [39]:
model.fit([X_train_input_ids, X_train_attention_masks], y_train_encoded, validation_data=([X_val_input_ids, X_val_attention_masks], y_val_encoded), epochs=3, batch_size=8)


Epoch 1/3


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


248/248 [==============================] - 354s 1s/step - loss: 0.2789 - accuracy: 0.8953 - val_loss: 0.0578 - val_accuracy: 0.9717
Epoch 2/3
248/248 [==============================] - 322s 1s/step - loss: 0.0569 - accuracy: 0.9818 - val_loss: 0.0335 - val_accuracy: 0.9960
Epoch 3/3
248/248 [==============================] - 341s 1s/step - loss: 0.0242 - accuracy: 0.9919 - val_loss: 0.0517 - val_accuracy: 0.9838


In [10]:
y_pred = np.argmax(model.predict(X_val_input_ids), axis=1)
print("Test Accuracy:", accuracy_score(y_val_encoded, y_pred))
print(classification_report(y_val_encoded, y_pred))

8/8 [==============================] - 11s 938ms/step
Test Accuracy: 0.1700404858299595
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.00      0.00      0.00        49
           2       0.00      0.00      0.00        66
           3       0.17      1.00      0.29        42
           4       0.00      0.00      0.00        39

    accuracy                           0.17       247
   macro avg       0.03      0.20      0.06       247
weighted avg       0.03      0.17      0.05       247



c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [11]:
# Assuming you have loaded the trained model and tokenizer
# model = loaded_model
# tokenizer = loaded_tokenizer

# Define a function to make predictions
def predict_disease(input_text):

    input_text = input_text.lower()
    # Tokenize the input text
    input_tokens = tokenizer(input_text, padding=True, truncation=True, return_tensors='tf')
    input_ids = input_tokens['input_ids']
    attention_mask = input_tokens['attention_mask']

    # Make predictions
    predictions = model.predict([input_ids, attention_mask])

    # Decode the predicted labels
    predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
    predicted_disease = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_disease

# Example usage:
input_text = "Struggling with severe abdominal pain and discomfort, accompanied by persistent vomiting and diarrhea."
predicted_disease = predict_disease(input_text)
print("Predicted Disease:", predicted_disease)


1/1 [==============================] - 2s 2s/step
Predicted Disease: Hypertension


In [43]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [ ]:
import pandas as pd

# Load the dataset
df_recommendations = pd.read_csv('shaheen\shaheen\datarecnew.csv')

# Let's assume your dataset has columns: 'Disease', 'Symptoms', and 'Recommendation'

# Create a dictionary to map diseases to recommendations
disease_recommendations = {}

# Iterate over the dataset to populate the dictionary
for index, row in df_recommendations.iterrows():
    disease = row['Disease']
    symptoms = row['Symptom']
    recommendation = row['Recommendation']

    if disease not in disease_recommendations:
        disease_recommendations[disease] = {'Symptoms': [], 'Recommendations': []}

    disease_recommendations[disease]['Symptoms'].append(symptoms)
    disease_recommendations[disease]['Recommendations'].append(recommendation)

from fuzzywuzzy import process

# Define a function to get recommendations for a given disease and provided symptoms
def get_recommendations_for_disease_and_symptoms(disease, provided_symptoms):
    print(disease, provided_symptoms)
    recommendations_info = disease_recommendations.get(disease)
    print(recommendations_info)
    if recommendations_info:
        symptoms = recommendations_info['Symptoms']
        recommendations = recommendations_info['Recommendations']

        # Join individual symptoms into a single string
        provided_symptoms_string = ' '.join(provided_symptoms)

        # Find closest matching symptom
        matched_symptom = process.extractOne(provided_symptoms_string, symptoms)[0]

        # Find index of matched symptom
        symptom_index = symptoms.index(matched_symptom)

        # Get recommendation corresponding to matched symptom
        recommendation = recommendations[symptom_index]

        return [recommendation]  # Return as list
    else:
        return None

# Integration example
# input_text = "heart attack symptoms chest pain"
predicted_disease_result = predict_disease(input_text)
# print("Predicted Disease:", predicted_disease_result)

# Get recommendations based on predicted disease and provided symptoms
provided_symptoms = input_text.split()
# print(provided_symptoms)  # Split input text into individual words
if predicted_disease_result:
    recommendations = get_recommendations_for_disease_and_symptoms(predicted_disease_result, provided_symptoms)
    if recommendations:
        print("Recommendations for", predicted_disease_result, ":", *recommendations)
    else:
        print("No recommendations available for the provided symptoms and disease.")
else:
    print("Unable to predict the disease.")

1/1 [==============================] - 0s 59ms/step
Hypertension ['Struggling', 'with', 'severe', 'abdominal', 'pain', 'and', 'discomfort,', 'accompanied', 'by', 'persistent', 'vomiting', 'and', 'diarrhea.']
{'Symptoms': ['I frequently experience weakness and trembling, particularly in the mornings.', 'I have a continual sensation of tightness in my muscles.', "I often feel like I'm in a haze or have difficulty focusing.", 'I consistently feel on edge and unable to unwind.', 'I frequently experience a mild ache or constriction in my chest.', 'I have an incessant urge to move my legs, particularly at night.', 'I often awaken with a throbbing headache in the morning.', 'I constantly feel dizzy or lightheaded.', 'I feel like my heart is racing or thumping in my chest.', 'I frequently experience a sensation of choking or a lump in my throat.', 'I have a persistent feeling of pressure or constriction in my chest.', 'I frequently feel irritable or angry without any apparent reason.', 'I cons

In [13]:
# Assuming you have loaded the trained model and tokenizer
# model = loaded_model
# tokenizer = loaded_tokenizer

# Define a function to make predictions
def predict_disease(input_text):

    input_text = input_text.lower()
    # Tokenize the input text
    input_tokens = tokenizer(input_text, padding=True, truncation=True, return_tensors='tf')
    input_ids = input_tokens['input_ids']
    attention_mask = input_tokens['attention_mask']

    # Make predictions
    predictions = model.predict([input_ids, attention_mask])

    # Decode the predicted labels
    predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
    predicted_disease = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_disease

# Example usage:
input_text = "i have nausea and chills, with abdominal cramps and vomiting."
predicted_disease = predict_disease(input_text)
print("Predicted Disease:", predicted_disease)

1/1 [==============================] - 2s 2s/step
Predicted Disease: Hypertension


In [2]:
%pip install googletrans
%pip install langid

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [54]:
pip install --upgrade googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17458 sha256=c787673553db068551891a3696c43f19cf407debc014b53a1788538b6ab56a7b
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\c0\59\9f\7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0
Note: you may need to restart the kernel to use updated packages.


In [14]:
from googletrans import Translator
import langid
from fuzzywuzzy import process
# Translator object for translation

translator = Translator()

# Function to translate text to English
def translate_to_english(text):
    translation = translator.translate(text, src='auto', dest='en')
    return translation.text

# Function to translate text to the original language
def translate_to_original_language(text, original_lang):
    translation = translator.translate(text, src='en', dest=original_lang)
    return translation.text

# Define a function to make predictions
def predict_disease(input_text):
    input_text_original = input_text
    print("Input Text (Original):", input_text_original)

    # Check if the input text is in English
    if input_text_original.isascii():
        # Input text is in English
        input_text_english = input_text_original
    else:
        # Input text is in non-English, translate to English
        input_text_english = translate_to_english(input_text_original)
        print("Input Text (English):", input_text_english)

    # Tokenize the input text
    input_tokens = tokenizer(input_text_english, padding=True, truncation=True, return_tensors='tf')
    input_ids = input_tokens['input_ids']
    attention_mask = input_tokens['attention_mask']

    # Make predictions
    predictions = model.predict([input_ids, attention_mask])

    # Decode the predicted labels
    predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
    predicted_disease = label_encoder.inverse_transform([predicted_class_index])[0]

    # If the input was in non-English, translate the prediction back to the original language
    if not input_text_original.isascii():
        original_lang = translator.detect(input_text_original).lang
        predicted_disease_1 = translate_to_original_language(predicted_disease, original_lang)
        print("Predicted Disease (Original Language):", predicted_disease_1)
    
    return predicted_disease

def translate_to_hindi(text):
    translator = Translator()
    translated_text = ''
    chunk_size = 500  # Adjust the chunk size as needed
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i+chunk_size]
        translation = translator.translate(chunk, dest='hi').text
        translated_text += translation + ' '
    translated_text = translated_text.strip()
    return translated_text

try:
    recommendation = translate_to_hindi(recommendation)
except Exception as e:
    print("Translation error:", e)
    recommendation = "Translation Error"
    
def get_default_message(language):
    if language == 'hi':
        return "कोई सुझाव उपलब्ध नहीं हैं"
    else:
        return "No recommendations available"

# Assuming the disease_recommendations dictionary is defined elsewhere

import pandas as pd

# Load the dataset
df_recommendations = pd.read_csv('datarecnew.csv')

# Let's assume your dataset has columns: 'Disease', 'Symptoms', and 'Recommendation'

# Create a dictionary to map diseases to recommendations
disease_recommendations = {}

# Iterate over the dataset to populate the dictionary
for index, row in df_recommendations.iterrows():
    disease = row['Disease']
    symptoms = row['Symptom']
    recommendation = row['Recommendation']

    if disease not in disease_recommendations:
        disease_recommendations[disease] = {'Symptoms': [], 'Recommendations': []}

    disease_recommendations[disease]['Symptoms'].append(symptoms)
    disease_recommendations[disease]['Recommendations'].append(recommendation)


# Define a function to get recommendations for a given disease and provided symptoms
def get_recommendations_for_disease_and_symptoms(disease, provided_symptoms, language='en'):
    print(disease, provided_symptoms)
    recommendations_info = disease_recommendations.get(disease)
    if recommendations_info:
        symptoms = recommendations_info['Symptoms']
        recommendations = recommendations_info['Recommendations']

        # Join individual symptoms into a single string
        provided_symptoms_string = ' '.join(provided_symptoms)

        # Find closest matching symptom
        matched_symptom = process.extractOne(provided_symptoms_string, symptoms)[0]

        # Find index of matched symptom
        symptom_index = symptoms.index(matched_symptom)

        # Get recommendation corresponding to matched symptom
        recommendation = recommendations[symptom_index]


        # Translate the recommendation to the detected language if it's not English
        if language != 'en':
            recommendation = translate_to_hindi(recommendation)

        return [recommendation]  # Return as list
    else:
        return None

# Function to detect language
def detect_language(text):
    return langid.classify(text)[0]

In [15]:
# Example usage:
input_text = "मैं अक्सर सुबह सूखे मुँह या गले के साथ उठता हूँ।."
predicted_disease= predict_disease(input_text)
print("Final Predicted Disease:",predicted_disease)

Input Text (Original): मैं अक्सर सुबह सूखे मुँह या गले के साथ उठता हूँ।.
Input Text (English): I often wake up with a dry mouth or throat in the morning.
1/1 [==============================] - 0s 60ms/step
Predicted Disease (Original Language): उच्च रक्तचाप
Final Predicted Disease: Hypertension


In [16]:
from fuzzywuzzy import process


# Function to translate text to Hindi

predicted_disease_result = predict_disease(input_text)
print("Predicted Disease:", predicted_disease_result)

# Determine the language of the input text
language = detect_language(input_text)
print("Detected Language:", language)

# Get recommendations based on predicted disease and provided symptoms
provided_symptoms = input_text.split()  # Split input text into individual words
if predicted_disease_result:
    recommendations = get_recommendations_for_disease_and_symptoms(predicted_disease_result, provided_symptoms, language)
    if recommendations:
        print("Recommendations for", predicted_disease_result, ":", *recommendations)
    else:
        print(get_default_message(language))
else:
    print("Unable to predict the disease.")

Input Text (Original): मैं अक्सर सुबह सूखे मुँह या गले के साथ उठता हूँ।.
Input Text (English): I often wake up with a dry mouth or throat in the morning.
1/1 [==============================] - 0s 61ms/step
Predicted Disease (Original Language): उच्च रक्तचाप
Predicted Disease: Hypertension
Detected Language: hi
Recommendations for Hypertension : पर्याप्त नींद सुनिश्चित करें और तनाव प्रबंधन तकनीकों का उपयोग करें।नियमित रूप से रक्तचाप की निगरानी करें और एक स्वास्थ्य सेवा पेशेवर से सलाह लें।


In [46]:
model.save('model')


INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [53]:
# # Define a function to get recommendations for a given disease and symptoms
# def get_recommendations_for_disease_and_symptoms(disease, provided_symptoms):
#     recommendations_info = disease_recommendations.get(disease)
#     if recommendations_info:
#         symptoms = recommendations_info['Symptoms']
#         recommendations = recommendations_info['Recommendations']

#         # Filter recommendations based on provided symptoms
#         filtered_recommendations = []
#         for symptom, recommendation in zip(symptoms, recommendations):
#             if all(s in symptom for s in provided_symptoms):
#                 filtered_recommendations.append(recommendation)

#         return filtered_recommendations
#     else:
#         return None

# # Integration example
# # input_text = "I frequently experience weakness and trembling, particularly in the mornings."
# predicted_disease_result = predict_disease(input_text)
# print("Predicted Disease:", predicted_disease_result)

# # Get recommendations based on predicted disease and provided symptoms
# provided_symptoms = input_text.split()  # Assuming symptoms are separated by spaces
# if predicted_disease_result:
#     recommendations = get_recommendations_for_disease_and_symptoms(predicted_disease_result, provided_symptoms)
#     if recommendations:
#         print("Recommendations for", predicted_disease_result, ":", *recommendations)
#     else:
#         print("No recommendations available for the provided symptoms and disease.")
# else:
#     print("Unable to predict the disease.")
import fuzzywuzzy
from fuzzywuzzy import process

# Define a function to get recommendations for a given disease and provided symptoms
# Define a function to get recommendations for a given disease and provided symptoms
# def get_recommendations_for_disease_and_symptoms(disease, provided_symptoms):
#     recommendations_info = disease_recommendations.get(disease)
#     if recommendations_info:
#         symptoms = recommendations_info['Symptoms']
#         recommendations = recommendations_info['Recommendations']

#         # Use fuzzy matching to find closest matching symptoms
#         matched_symptoms = [process.extractOne(symptom, provided_symptoms)[0] for symptom in symptoms]

#         # Initialize a set to store unique recommendations
#         unique_recommendations = set()

#         # Filter recommendations based on matched symptoms and add unique recommendations to the set
#         for matched_symptom, recommendation in zip(matched_symptoms, recommendations):
#             if matched_symptom in provided_symptoms:
#                 unique_recommendations.add(recommendation)

#         return list(unique_recommendations)  # Convert set to list before returning
#     else:
#         return None

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFDistilBertModel, DistilBertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import DistilBertTokenizer

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load the dataset
df = pd.read_csv('shaheen\shaheen\data.csv')

df= df.sample(frac=1)
df['Symptoms'] = df['Symptoms'].str.lower()
df['Medical_History'] = df['Medical_History'].str.lower()

X_text = df['Symptoms'] + " " + df['Medical_History']
y = df['Disease']

# Tokenize input data
X_tokenized = tokenizer(X_text.tolist(), padding=True, truncation=True, return_tensors='tf')

# Convert tensors to NumPy arrays
X_input_ids = X_tokenized['input_ids'].numpy()
X_attention_masks = X_tokenized['attention_mask'].numpy()

# Split the dataset into train, validation, and test sets
X_train_input_ids, X_temp_input_ids, X_train_attention_masks, X_temp_attention_masks, y_train, y_temp = train_test_split(X_input_ids, X_attention_masks, y, test_size=0.2, random_state=42)
X_val_input_ids, X_test_input_ids, X_val_attention_masks, X_test_attention_masks, y_val, y_test = train_test_split(X_temp_input_ids, X_temp_attention_masks, y_temp, test_size=0.5, random_state=42)

class DiseaseClassifier(tf.keras.Model):
    def __init__(self, num_classes):
        super(DiseaseClassifier, self).__init__()
        self.distilbert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.dense = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        outputs = self.distilbert(inputs)[0]
        pooled_output = outputs[:, 0, :]  # take [CLS] token
        pooled_output = self.dropout(pooled_output)
        logits = self.dense(pooled_output)
        return logits

num_classes = len(df['Disease'].unique())
model = DiseaseClassifier(num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

from sklearn.preprocessing import LabelEncoder

# Encode target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
print(y_val_encoded)

model.fit([X_train_input_ids, X_train_attention_masks], y_train_encoded, validation_data=([X_val_input_ids, X_val_attention_masks], y_val_encoded), epochs=1, batch_size=8)
y_pred = np.argmax(model.predict(X_val_input_ids), axis=1)
print("Test Accuracy:", accuracy_score(y_val_encoded, y_pred))
print(classification_report(y_val_encoded, y_pred))


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


[1 0 0 1 0 2 2 2 1 0 2 2 2 0 1 2 1 2 0 2 2 2 0 1 2 2 0 0 2 1 0 0 2 1 0 2 0
 1 1 1 0 0 0 1 0 0 2 1 1 1 0 2 0 1 0 1 0 2 2 2 0 2 1 1 2 0 0 0 0 1 2 1 1 1
 1 0 1 1 1 1 1 2 0 1 2 1 0 2 0 0 0 1 2 2 2 1 2 2 1 2 2 1 0 2 1 2 1 1 0 1 2
 1 0 1 0 1 0 2 2 1 1 1 0 1 2 1 2 2 1 0 0 0 2 1 1 2 1 0 0 2 0 2 0 1 1]


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 15/146 [==>...........................] - ETA: 2:56 - loss: 1.0238 - accuracy: 0.4917

KeyboardInterrupt: 